In [4]:
#test dataset
!wget https://github.com/google-research/google-research/raw/master/goemotions/data/test.tsv


--2023-01-19 22:13:49--  https://github.com/google-research/google-research/raw/master/goemotions/data/test.tsv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv [following]
--2023-01-19 22:13:49--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/test.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 436706 (426K) [text/plain]
Saving to: ‘test.tsv’

test.tsv            100%[===================>] 426.47K  --.-KB/s    in 0.03s   

2023-01-19 22:13:49 (11.9 MB/s) - ‘test.tsv’ saved [436706/43670

In [5]:
#labels
!wget https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt

--2023-01-19 22:20:14--  https://raw.githubusercontent.com/google-research/google-research/master/goemotions/data/emotions.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248 [text/plain]
Saving to: ‘emotions.txt’

emotions.txt        100%[===================>]     248  --.-KB/s    in 0s      

2023-01-19 22:20:15 (15.3 MB/s) - ‘emotions.txt’ saved [248/248]



In [41]:
import pandas as pd
labels_df = pd.read_csv('emotions.txt', header=None)
labels = labels_df.values[:,0].tolist()

In [42]:
data_df = pd.read_csv('test.tsv', sep='\t', header=None)

In [62]:
def make_prompt_answer(txt, txt_labels):
  label_items = list(map(labels.__getitem__, map(int,txt_labels.split(','))))
  label_len = len(label_items)
  prompt = "From the given list of all emotions, choose the ones that the input text arouses in most people reading it. Write your answer in the form of a Python list containing exactly {} selected most matching {}. List of all emotions: admiration, amusement, anger, annoyance, approval, caring, confusion, curiosity, desire, disappointment, disapproval, disgust, embarrassment, excitement, fear, gratitude, grief, joy, love, nervousness, optimism, pride, realization, relief, remorse, sadness, surprise, neutral. Input text: {}".format(
      label_len, 'emotion' if label_len==1 else 'emotions', txt)
  return "{}\t{}".format(prompt, ','.join(label_items))
    

In [63]:
output_items = []
for index, (txt, txt_labels, annotator) in data_df.iterrows():
  output_items.append(make_prompt_answer(txt, txt_labels))


In [64]:
with open('goemotions.tsv', 'w') as f:
  f.write('\n'.join(output_items[:500]))